In [1]:
!pip install torch
!pip install torch-geometric
!pip install networkx
!pip install matplotlib


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


### 1) GNN 노드-엣지 그래프 학습

In [34]:
### 1) 단순 유사도 기반

In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import RandomLinkSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json

# JSON 파일에서 데이터셋 로드
with open('data/gnn_enhanced_700_dict.json', 'r') as f:
    dataset = json.load(f)

# 노드와 엣지 추출
nodes = dataset['nodes']
edges = dataset['edges']

# 노드 특징 생성 (임시로 단위 행렬 사용)
node_features = torch.eye(len(nodes), dtype=torch.float)

# 엣지 리스트 생성
edge_index = torch.tensor([[edge['source'] - 1, edge['target'] - 1] for edge in edges], dtype=torch.long).t().contiguous()

# 데이터 객체 생성
data = Data(x=node_features, edge_index=edge_index)

# 데이터셋 분할 (훈련, 검증, 테스트)
transform = RandomLinkSplit(is_undirected=True)
train_data, val_data, test_data = transform(data)

print("train_data : ", train_data)
print("val_data :", val_data)
print("test_data : ", test_data)

train_data :  Data(x=[1052, 1052], edge_index=[2, 1122], edge_label=[1122], edge_label_index=[2, 1122])
val_data : Data(x=[1052, 1052], edge_index=[2, 1122], edge_label=[158], edge_label_index=[2, 158])
test_data :  Data(x=[1052, 1052], edge_index=[2, 1280], edge_label=[318], edge_label_index=[2, 318])


In [11]:
import json
import torch
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, GCNConv, Linear
from torch_geometric.data import HeteroData

# 데이터 로드
with open('data/gnn_enhanced_700_dict.json', 'r') as f:
    dataset = json.load(f)

# HeteroData 객체 초기화
data = HeteroData()

# 노드 임베딩 생성 (예시로 랜덤 임베딩 사용)
num_nodes = len(dataset['nodes'])
embedding_dim = 64
node_embeddings = torch.randn((num_nodes, embedding_dim))
data['word'].x = node_embeddings

# 엣지 정보 생성
# 'related_to' 관계를 가진 엣지를 구축
source_nodes = []
target_nodes = []
for edge in dataset.get('edges', []):
    source_nodes.append(edge['source'])
    target_nodes.append(edge['target'])

# edge_index를 생성 (2xN 형태의 텐서)
edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
data['word', 'related_to', 'word'].edge_index = edge_index

# Heterogeneous GNN 모델 정의
class HeteroGNN(torch.nn.Module):
    def __init__(self, metadata, in_channels, hidden_channels, out_channels):
        super(HeteroGNN, self).__init__()
        self.conv1 = HeteroConv({
            ('word', 'related_to', 'word'): GCNConv(in_channels, hidden_channels)
        }, aggr='mean')
        
        self.conv2 = HeteroConv({
            ('word', 'related_to', 'word'): GCNConv(hidden_channels, out_channels)
        }, aggr='mean')

        # Fully connected layer for final output
        self.fc = Linear(out_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        # 첫 번째 계층
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: F.relu(x) for key, x in x_dict.items()}
        
        # 두 번째 계층
        x_dict = self.conv2(x_dict, edge_index_dict)
        x_dict = {key: F.relu(x) for key, x in x_dict.items()}
        
        # 'word' 노드에 대해 완전 연결 계층 적용
        x_word = self.fc(x_dict['word'])
        return x_word

# 모델, 손실 함수 및 옵티마이저 초기화
in_channels = data['word'].x.shape[1]
model = HeteroGNN(metadata=data.metadata(), in_channels=in_channels, hidden_channels=32, out_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# 훈련 함수
def train():
    model.train()
    optimizer.zero_grad()
    
    out = model(data.x_dict, data.edge_index_dict)
    target = torch.randint(0, 2, (num_nodes,))  # 임의 이진 레이블 생성
    loss = loss_fn(out, target)
    
    loss.backward()
    optimizer.step()
    
    return loss.item()

# 평가 함수
def test():
    model.eval()
    with torch.no_grad():
        out = model(data.x_dict, data.edge_index_dict)
        pred = out.argmax(dim=-1)
        target = torch.randint(0, 2, (num_nodes,))  # 테스트용 임의 이진 레이블
        accuracy = (pred == target).sum().item() / target.size(0)
        return accuracy

# 훈련 루프
for epoch in range(1, 101):
    loss = train()
    if epoch % 10 == 0:
        accuracy = test()
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')


RuntimeError: index 1052 is out of bounds for dimension 0 with size 1052

In [ ]:
# GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# 모델, 옵티마이저, 손실 함수 초기화
model = GCN(in_channels=node_features.size(1), out_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCELoss()

# 훈련 함수
def train():
    model.train()
    optimizer.zero_grad()

    emb = model(train_data.x, train_data.edge_index)
    src, dest = train_data.edge_label_index
    pred = (emb[src] * emb[dest]).sum(dim=-1).sigmoid()

    target = train_data.edge_label.float()
    loss = loss_fn(pred, target)

    loss.backward()
    optimizer.step()
    
    return loss.item()

# 평가 함수
def test(data):
    model.eval()
    with torch.no_grad():
        emb = model(data.x, data.edge_index)
        src, dest = data.edge_label_index
        pred = (emb[src] * emb[dest]).sum(dim=-1).sigmoid()
        pred_label = (pred > 0.5).float()
        target = data.edge_label.float()
        
        # 평가 지표 : 정확도, 정밀도, 재현율, f1-score
        accuracy = accuracy_score(target.cpu(), pred_label.cpu())
        precision = precision_score(target.cpu(), pred_label.cpu(), zero_division=0)
        recall = recall_score(target.cpu(), pred_label.cpu(), zero_division=0)
        f1 = f1_score(target.cpu(), pred_label.cpu(), zero_division=0)

        return accuracy, precision, recall, f1

# 훈련 루프
for epoch in range(1, 101):
    loss = train()
    if epoch % 10 == 0:
        accuracy, precision, recall, f1 = test(test_data)
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')


Epoch 10, Loss: 0.5460, Accuracy: 0.5157, Precision: 0.5095, Recall: 0.8428, F1: 0.6351
Epoch 20, Loss: 0.1978, Accuracy: 0.5252, Precision: 0.5211, Recall: 0.6226, F1: 0.5673
Epoch 30, Loss: 0.0590, Accuracy: 0.5094, Precision: 0.5086, Recall: 0.5597, F1: 0.5329
Epoch 40, Loss: 0.0147, Accuracy: 0.5314, Precision: 0.5263, Recall: 0.6289, F1: 0.5731
Epoch 50, Loss: 0.0035, Accuracy: 0.5283, Precision: 0.5243, Recall: 0.6101, F1: 0.5640
Epoch 60, Loss: 0.0010, Accuracy: 0.5314, Precision: 0.5269, Recall: 0.6164, F1: 0.5681
Epoch 70, Loss: 0.0004, Accuracy: 0.5314, Precision: 0.5263, Recall: 0.6289, F1: 0.5731
Epoch 80, Loss: 0.0002, Accuracy: 0.5346, Precision: 0.5288, Recall: 0.6352, F1: 0.5771
Epoch 90, Loss: 0.0002, Accuracy: 0.5314, Precision: 0.5263, Recall: 0.6289, F1: 0.5731
Epoch 100, Loss: 0.0001, Accuracy: 0.5314, Precision: 0.5263, Recall: 0.6289, F1: 0.5731


In [35]:
### 2) 관계 유형 추가
## => 그 관계를 어떻게 할 것인지 정의할 필요가 있는 듯함. 

## 예시 형태)
# 포함 관계 (Inclusion): 하나의 단어가 다른 단어를 포함하거나 소속된 관계.
# 종속 관계 (Dependency): 하나의 단어가 다른 단어에 의존하는 관계.
# 인과 관계 (Causal): 한 단어가 다른 단어의 원인이나 결과로 작용하는 관계.
# 상관 관계 (Correlation): 두 단어가 서로 상관관계를 가지는 경우.
# 구조적 관계 (Structural): 두 단어가 어떤 구조 내에서 관계를 맺고 있는 경우.
# 기능적 관계 (Functional): 특정 기능을 수행하기 위해 두 단어가 연결된 경우.
# 맥락적 관계 (Contextual): 특정 문맥에서 두 단어가 함께 사용되는 관계.

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Embedding
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json
from torch_geometric.transforms import RandomLinkSplit

# Load dataset from JSON file
with open('data/gnn_enhanced_700_dict.json', 'r') as f:
    dataset = json.load(f)

# Extract nodes and edges
nodes = dataset['nodes']
edges = dataset['edges']

# Create node features (identity matrix for now)
node_features = torch.eye(len(nodes), dtype=torch.float)

# Create relation description mapping -> 여기서 relation_description은 각 단어, 관계 단어 간의 정의에 가까움. 
relation_descriptions = list(set(edge['relation_description'] for edge in edges))

## => enhanced_dict에 정의된 relation_description의 관계가 너무 많아서 학습이 안됨. 
relation_map = {desc: idx for idx, desc in enumerate(relation_descriptions)}

# Create edge_index and edge_type
edge_index = []
edge_type = []
for edge in edges:
    edge_index.append([edge['source'] - 1, edge['target'] - 1])
    edge_type.append(relation_map[edge['relation_description']])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_type = torch.tensor(edge_type, dtype=torch.long)

# Create a data object
data = Data(x=node_features, edge_index=edge_index, edge_type=edge_type)

# Split data using RandomLinkSplit with negative samples
transform = RandomLinkSplit(num_val=0.1, num_test=0.1, is_undirected=True, add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

# Define GCN model with relation embedding
class GCNWithRelations(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_relations):
        super(GCNWithRelations, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)
        self.relation_embed = Embedding(num_relations, 16)  # Embedding for relation types

    def forward(self, x, edge_index, edge_type):
        edge_attr = self.relation_embed(edge_type)  # Embed relation types
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize model, optimizer, and loss function
model = GCNWithRelations(in_channels=node_features.size(1), out_channels=16, num_relations=len(relation_map))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCELoss()

# Training function
def train():
    model.train()
    optimizer.zero_grad()
    emb = model(train_data.x, train_data.edge_index, train_data.edge_type)
    src, dest = train_data.edge_label_index
    pred = (emb[src] * emb[dest]).sum(dim=-1).sigmoid()
    loss = loss_fn(pred, train_data.edge_label.float())
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation function
def test(data):
    model.eval()
    with torch.no_grad():
        emb = model(data.x, data.edge_index, data.edge_type)
        src, dest = data.edge_label_index
        pred = (emb[src] * emb[dest]).sum(dim=-1).sigmoid()
        pred_label = (pred > 0.5).float()
        target = data.edge_label.float()
        accuracy = accuracy_score(target.cpu(), pred_label.cpu())
        precision = precision_score(target.cpu(), pred_label.cpu(), zero_division=0)
        recall = recall_score(target.cpu(), pred_label.cpu(), zero_division=0)
        f1 = f1_score(target.cpu(), pred_label.cpu(), zero_division=0)
        return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 101):
    loss = train()
    if epoch % 10 == 0:
        accuracy, precision, recall, f1 = test(test_data)
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

# Print label distribution
print(f"Train positive labels: {train_data.edge_label.sum().item()}, Train total labels: {train_data.edge_label.size(0)}")
print(f"Test positive labels: {test_data.edge_label.sum().item()}, Test total labels: {test_data.edge_label.size(0)}")


Epoch 10, Loss: 0.5628, Accuracy: 0.5190, Precision: 0.5099, Recall: 0.9747, F1: 0.6696
Epoch 20, Loss: 0.2197, Accuracy: 0.5316, Precision: 0.5229, Recall: 0.7215, F1: 0.6064
Epoch 30, Loss: 0.0567, Accuracy: 0.5759, Precision: 0.5588, Recall: 0.7215, F1: 0.6298
Epoch 40, Loss: 0.0124, Accuracy: 0.5823, Precision: 0.5670, Recall: 0.6962, F1: 0.6250
Epoch 50, Loss: 0.0030, Accuracy: 0.5696, Precision: 0.5556, Recall: 0.6962, F1: 0.6180
Epoch 60, Loss: 0.0008, Accuracy: 0.5696, Precision: 0.5556, Recall: 0.6962, F1: 0.6180
Epoch 70, Loss: 0.0003, Accuracy: 0.5633, Precision: 0.5510, Recall: 0.6835, F1: 0.6102
Epoch 80, Loss: 0.0002, Accuracy: 0.5633, Precision: 0.5510, Recall: 0.6835, F1: 0.6102
Epoch 90, Loss: 0.0001, Accuracy: 0.5633, Precision: 0.5510, Recall: 0.6835, F1: 0.6102
Epoch 100, Loss: 0.0001, Accuracy: 0.5633, Precision: 0.5510, Recall: 0.6835, F1: 0.6102
Train positive labels: 641.0, Train total labels: 1282
Test positive labels: 79.0, Test total labels: 158


### 2) 새로운 경제 키워드 입력 시 엣지(관계) 추론

In [ ]:
### 1) 단순 유사도 기반 그래프에서 새로운 키워드(ex. 경제성장률)를 추가했을 때

In [45]:
import torch

# 새로운 키워드 추가 (예: "경제성장률")
new_keyword = "경제성장률"
new_node_index = len(nodes)  # 새로운 노드의 인덱스

# 노드 리스트에 새로운 키워드 추가
nodes.append({"id": new_node_index + 1, "word": new_keyword})

# 노드 특징 매트릭스 확장 (기존 노드 + 새로운 노드)
new_node_features = torch.eye(len(nodes), dtype=torch.float)

# 데이터 업데이트
data.x = new_node_features
data.num_nodes = len(nodes)

# 모델 재정의
model = GCN(in_channels=new_node_features.size(1), out_channels=16)  # 다시 모델 초기화

# 새로운 노드와 기존 노드 간의 관계 예측
model.eval()
with torch.no_grad():
    emb = model(data.x, data.edge_index)
    new_node_emb = emb[new_node_index]
    
    # 기존 모든 노드와 새로운 노드 간의 유사도 계산
    pred_edges = []
    for i in range(len(nodes) - 1):
        existing_node_emb = emb[i]
        similarity = (new_node_emb * existing_node_emb).sum().sigmoid().item()
        pred_edges.append((i + 1, new_node_index + 1, similarity))

# 예측된 관계 출력
pred_edges.sort(key=lambda x: x[2], reverse=True)  # 유사도에 따라 정렬
for edge in pred_edges:
    print(f"노드 {edge[0]}와 노드 {edge[1]} 사이의 예측된 관계 유사도: {edge[2]:.4f}")



노드 882와 노드 1055 사이의 예측된 관계 유사도: 0.5026
노드 299와 노드 1055 사이의 예측된 관계 유사도: 0.5024
노드 274와 노드 1055 사이의 예측된 관계 유사도: 0.5023
노드 298와 노드 1055 사이의 예측된 관계 유사도: 0.5023
노드 338와 노드 1055 사이의 예측된 관계 유사도: 0.5023
노드 655와 노드 1055 사이의 예측된 관계 유사도: 0.5022
노드 50와 노드 1055 사이의 예측된 관계 유사도: 0.5022
노드 300와 노드 1055 사이의 예측된 관계 유사도: 0.5021
노드 573와 노드 1055 사이의 예측된 관계 유사도: 0.5021
노드 877와 노드 1055 사이의 예측된 관계 유사도: 0.5021
노드 93와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 654와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 532와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 214와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 308와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 856와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 689와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 216와 노드 1055 사이의 예측된 관계 유사도: 0.5020
노드 797와 노드 1055 사이의 예측된 관계 유사도: 0.5019
노드 533와 노드 1055 사이의 예측된 관계 유사도: 0.5019
노드 611와 노드 1055 사이의 예측된 관계 유사도: 0.5019
노드 612와 노드 1055 사이의 예측된 관계 유사도: 0.5019
노드 339와 노드 1055 사이의 예측된 관계 유사도: 0.5019
노드 531와 노드 1055 사이의 예측된 관계 유사도: 0.5018
노드 613와 노드 1055 사이의 예측된 관계 유사도: 0.5018
노드 455와 노드 1055 사이의 예측된 관계 

In [48]:
### 2) 새로운 관계 유형이 들어갔을 때

## => 데이터셋의 relation 규명 부분에서 수정이 필요해보임. (ex. 포함 관계, 종속 관계, )

{'NotebookApp': {'iopub_data_rate_limit': 10000000, 'rate_limit_window': 3.0}}

In [50]:
import torch
from torch.nn import Embedding
from torch_geometric.nn import GCNConv

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {
    'NotebookApp': {
        'iopub_data_rate_limit': 10000000,  # 원하는 출력 제한 크기 설정 (예: 10MB)
        'rate_limit_window': 3.0            # 시간 윈도우 (초)
    }
})

# 새로운 키워드 추가 (예: "경제성장률")
new_keyword = "경제성장률"
new_node_index = len(nodes)  # 새로운 노드의 인덱스

# 노드 리스트에 새로운 키워드 추가
nodes.append({"id": new_node_index + 1, "word": new_keyword})

# 노드 특징 매트릭스 확장 (기존 노드 + 새로운 노드)
new_node_features = torch.eye(len(nodes), dtype=torch.float)

# 데이터 업데이트
data.x = new_node_features
data.num_nodes = len(nodes)

# 관계 설명 매핑 및 관계 임베딩 추가
relation_descriptions = list(set(edge['relation_description'] for edge in edges))
relation_map = {desc: idx for idx, desc in enumerate(relation_descriptions)}
num_relations = len(relation_map)

# GCN 모델 정의 (관계 임베딩 포함)
class GCNWithRelations(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_relations):
        super(GCNWithRelations, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)
        self.relation_embed = Embedding(num_relations, 16)  # Relation type embedding

    def forward(self, x, edge_index, edge_type):
        edge_attr = self.relation_embed(edge_type)  # Embed relation types
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x, edge_attr

# 모델 초기화
model = GCNWithRelations(in_channels=new_node_features.size(1), out_channels=16, num_relations=num_relations)

# 기존 edge_index에 관계 유형 추가
edge_index = torch.tensor([[edge['source'] - 1, edge['target'] - 1] for edge in edges], dtype=torch.long).t().contiguous()
edge_type = torch.tensor([relation_map[edge['relation_description']] for edge in edges], dtype=torch.long)

# 모델 평가 및 관계 예측
model.eval()
with torch.no_grad():
    emb, relation_emb = model(data.x, edge_index, edge_type)
    new_node_emb = emb[new_node_index]
    
    # 기존 모든 노드와 새로운 노드 간의 관계 예측
    pred_edges = []
    for i in range(len(nodes) - 1):
        existing_node_emb = emb[i]
        for relation_id in range(num_relations):
            relation_emb_vec = relation_emb[relation_id]
            # Adjust similarity by relation type
            similarity = (new_node_emb * existing_node_emb * relation_emb_vec).sum().sigmoid().item()
            pred_edges.append((i + 1, new_node_index + 1, relation_descriptions[relation_id], similarity))

# 예측된 관계 출력
pred_edges.sort(key=lambda x: x[3], reverse=True)  # 유사도에 따라 정렬
for edge in pred_edges:
    print(f"노드 {edge[0]}와 노드 {edge[1]} 사이의 예측된 관계 ({edge[2]}) 유사도: {edge[3]:.4f}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
### 시각화 예시

In [51]:
import networkx as nx
import plotly.graph_objs as go
from matplotlib import font_manager, rc

# 한글 폰트 설정
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"  # 시스템에 설치된 한글 글꼴 경로
font_name = font_manager.FontProperties(fname=font_path).get_name()

# 기존 그래프에 새 노드와 엣지 추가
G = nx.Graph()

# 기존 노드 추가
for node in nodes:
    G.add_node(node['id'], label=node['word'])

# 기존 엣지 추가
for edge in data.edge_index.t().tolist():
    G.add_edge(edge[0] + 1, edge[1] + 1)

# 예측된 엣지 추가 (임계값 0.5 이상만 추가)
threshold = 0.5
for src, dst, similarity in pred_edges:
    if similarity >= threshold:
        G.add_edge(src, dst, weight=similarity)

# 노드와 엣지의 위치 설정
pos = nx.spring_layout(G, seed=42)

# Plotly 노드 설정
edge_x = []
edge_y = []
edge_weights = []  # 엣지 가중치 저장

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_weights.append(edge[2].get('weight', 0.5))  # 기본 가중치 0.5

# 엣지 시각화
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines')

# 엣지 레이블 표시
edge_texts = []
for edge in G.edges(data=True):
    edge_texts.append(f"Similarity: {edge[2].get('weight', 0.5):.2f}")

edge_label_trace = go.Scatter(
    x=edge_x[::3],  # 엣지 중간 지점에 레이블 배치
    y=edge_y[::3],
    text=edge_texts,
    mode='text',
    textposition="top center",
    hoverinfo='none'
)

node_x = []
node_y = []
node_text = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(G.nodes[node]['label'])

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text=node_text,
    textposition="top center",
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
    )
)

# 대화형 그래프 생성
fig = go.Figure(data=[edge_trace, edge_label_trace, node_trace],
                layout=go.Layout(
                    title="경제 키워드 지식 그래프",
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="Plotly를 사용한 대화형 그래프",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002)],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)))

# 그래프를 HTML 파일로 저장
fig.write_html("economic_keyword_graph.html")

# Chrome 브라우저로 열기
webbrowser.get('chrome').open("economic_keyword_graph.html")


ValueError: too many values to unpack (expected 3)